In [2]:
import requests
import json
import time

all_products = []
page_number = 1
page_limit = 250 # Shopify API default limit

while True:
    print(f"Fetching page {page_number}...")
    url = f"https://designerdaydream.com/products.json?page={page_number}&limit={page_limit}"

    try:
        res = requests.get(url)
        res.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = res.json()
        products = data.get('products', [])

        all_products.extend(products)

        if len(products) < page_limit:
            print(f"Found {len(products)} products on page {page_number}. Assuming this is the last page.")
            break

        print(f"Found {len(products)} products on page {page_number}.")
        page_number += 1
        time.sleep(0.5) # Be kind to the server

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        break
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {url}. Response content: {res.text[:200]}...")
        break

print(f"\nTotal products scraped: {len(all_products)}")

# Optionally, save to a JSON file
file_path = 'designerdaydream.com_products.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(all_products, f, indent=4)
print(f"All products saved to {file_path}")

Fetching page 1...
Found 250 products on page 1.
Fetching page 2...
Found 250 products on page 2.
Fetching page 3...
Found 250 products on page 3.
Fetching page 4...
Found 250 products on page 4.
Fetching page 5...
Found 250 products on page 5.
Fetching page 6...
Found 250 products on page 6.
Fetching page 7...
Found 250 products on page 7.
Fetching page 8...
Found 60 products on page 8. Assuming this is the last page.

Total products scraped: 1810
All products saved to designerdaydream.com_products.json


In [8]:
pip install json5

In [9]:
import requests
import json5
from bs4 import BeautifulSoup


def extract_js_object(text, key):
    start = text.find(f"{key}:")
    if start == -1:
        return None

    brace_start = text.find("{", start)
    depth = 0

    for i in range(brace_start, len(text)):
        if text[i] == "{":
            depth += 1
        elif text[i] == "}":
            depth -= 1
            if depth == 0:
                return text[brace_start:i + 1]

    return None


url = "https://designerdaydream.com/products/alexander-mcqueen-am0260o-gold-metal-aviator-frames"
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")

script_text = next(
    s.string for s in soup.find_all("script")
    if s.string and "window.DCART" in s.string and "barcode" in s.string
)

product_js = extract_js_object(script_text, "product")

# ✅ Parse JS safely
product_data = json5.loads(product_js)

print(product_data["title"])
print(product_data["variants"][0]["barcode"])

Alexander McQueen AM0260O Gold Metal Aviator Frames
889652283975


In [13]:
import requests
import json5
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed


# =========================
# CONFIG
# =========================

BASE_URL = "https://designerdaydream.com/products/"
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}
TIMEOUT = 20
WORKERS = 10


# =========================
# HELPERS
# =========================

def extract_js_object(text, key):
    """
    Extract JS object like: product: { ... }
    using brace counting (safe for nested objects)
    """
    start = text.find(f"{key}:")
    if start == -1:
        return None

    brace_start = text.find("{", start)
    if brace_start == -1:
        return None

    depth = 0
    for i in range(brace_start, len(text)):
        if text[i] == "{":
            depth += 1
        elif text[i] == "}":
            depth -= 1
            if depth == 0:
                return text[brace_start:i + 1]

    return None


# =========================
# PRIMARY: Shopify product.js
# =========================

def get_product_js(handle):
    """
    Fetch clean Shopify JSON:
    /products/{handle}.js
    """
    url = f"{BASE_URL}{handle}.js"
    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)

    if r.status_code != 200:
        raise RuntimeError(f"product.js HTTP {r.status_code}")

    if not r.headers.get("content-type", "").startswith("application/json"):
        raise RuntimeError("product.js not JSON")

    data = r.json()

    # Safety check
    if data.get("handle") != handle:
        raise RuntimeError("product.js handle mismatch")

    return {
        "url": url,
        "status": "success",
        "source": "product.js",
        "data": data
    }


# =========================
# FALLBACK: DCART parsing
# =========================

def get_product_dcart(handle):
    """
    Fallback parser for broken Shopify themes using window.DCART
    """
    url = f"{BASE_URL}{handle}"
    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    candidates = []

    for s in soup.find_all("script"):
        text = s.string
        if not text:
            continue
        if "window.DCART" not in text:
            continue
        if "product:" not in text:
            continue

        product_js = extract_js_object(text, "product")
        if not product_js:
            continue

        try:
            product = json5.loads(product_js)
            candidates.append(product)
        except Exception:
            continue

    if not candidates:
        raise RuntimeError("DCART product not found")

    # Pick correct product by handle
    for product in candidates:
        if product.get("handle") == handle:
            return {
                "url": url,
                "status": "success",
                "source": "dcart",
                "data": product
            }

    raise RuntimeError(
        f"DCART mismatch (found {[p.get('handle') for p in candidates]})"
    )


# =========================
# UNIFIED FETCH
# =========================

def fetch_product(product):
    """
    product: dict containing at least { "handle": ... }
    """
    handle = product["handle"]

    # 1) Try clean JSON endpoint
    try:
        return get_product_js(handle)
    except Exception:
        pass

    # 2) Fallback to DCART
    return get_product_dcart(handle)


# =========================
# CONCURRENT RUNNER
# =========================

def fetch_all_products(all_products, workers=WORKERS):
    results = []

    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = {
            executor.submit(fetch_product, p): p["handle"]
            for p in all_products
            if "handle" in p
        }

        for future in as_completed(futures):
            handle = futures[future]
            try:
                result = future.result()
                results.append(result)
                print(f"✅ {handle} ({result['source']})")
            except Exception as e:
                print(f"❌ {handle} → {e}")

    return results

In [15]:
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed


# =========================
# CONFIG
# =========================

BASE_URL = "https://designerdaydream.com/products/"
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}
TIMEOUT = 20
WORKERS = 10


# =========================
# PRODUCT.JS FETCHER
# =========================

def get_product_js(handle):
    """
    Fetch Shopify product JSON via:
    /products/{handle}.js
    """
    url = f"{BASE_URL}{handle}.js"

    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)

    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}")

    if not r.headers.get("content-type", "").startswith("application/json"):
        raise RuntimeError("Not JSON response")

    data = r.json()

    # Safety check
    if data.get("handle") != handle:
        raise RuntimeError(
            f"Handle mismatch (expected {handle}, got {data.get('handle')})"
        )

    return {
        "url": url,
        "status": "success",
        "data": data
    }


# =========================
# UNIFIED FETCH
# =========================

def fetch_product(product):
    """
    product must contain: { "handle": ... }
    """
    handle = product["handle"]
    return get_product_js(handle)


# =========================
# CONCURRENT RUNNER
# =========================

def fetch_all_products(all_products, workers=WORKERS):
    results = []

    with ThreadPoolExecutor(max_workers=workers) as executor:
        futures = {
            executor.submit(fetch_product, p): p["handle"]
            for p in all_products
            if "handle" in p
        }

        for future in as_completed(futures):
            handle = futures[future]
            try:
                result = future.result()
                results.append(result)
                print(f"✅ {handle}")
            except Exception as e:
                print(f"❌ {handle} → {e}")

    return results


# =========================
# EXAMPLE USAGE
# =========================

if __name__ == "__main__":
    products_data = fetch_all_products(all_products, workers=10)

    print("\nFetched:", len(products_data), "products")

❌ loewe-lw40156u-cat-eye-sunglasses-in-brown → Not JSON response
❌ loewe-lw40127u-anagram-sunglasses-in-black → Not JSON response
❌ celine-cl4006in-thin-shadow-sunglasses-in-blue → Not JSON response
❌ loewe-lw40127i-anagram-sunglasses-in-black → Not JSON response
❌ loewe-lw50068i-anagram-round-frames-in-transparent-brown → Not JSON response
❌ loewe-lw40154u-anagram-sunglasses-in-black → Not JSON response
❌ loewe-lw50069i-anagram-frames-in-black-horn → Not JSON response
❌ loewe-lw50068i-anagram-round-frames-in-black → Not JSON response
❌ loewe-lw50076i-anagram-frames-in-black → Not JSON response
❌ loewe-lw50038u-anagram-frames-in-gold → Not JSON response
❌ loewe-lw50076i-anagram-eyeglasses-frames-in-beige → Not JSON response
❌ loewe-lw40115u-cat-eye-sunglasses-in-gold-mirror → Not JSON response
❌ loewe-lw40102f-anagram-sunglasses-in-dark-havana-brown → Not JSON response
❌ loewe-lw50069i-anagram-frames-in-blonde-havana → Not JSON response
❌ loewe-lw40134u-oversized-square-sunglasses-in-h

In [17]:
import asyncio
import aiohttp


# =========================
# CONFIG
# =========================

BASE_URL = "https://designerdaydream.com/products/"
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}
TIMEOUT = aiohttp.ClientTimeout(total=20)
CONCURRENCY = 10


# =========================
# PRODUCT.JS FETCHER
# =========================

async def get_product_js(session, handle, semaphore):
    url = f"{BASE_URL}{handle}.js"

    async with semaphore:
        async with session.get(url) as resp:
            if resp.status != 200:
                raise RuntimeError(f"HTTP {resp.status}")

            content_type = resp.headers.get("Content-Type", "")
            if not content_type.startswith("application/json"):
                raise RuntimeError("Not JSON response")

            data = await resp.json()

            # Safety check
            if data.get("handle") != handle:
                raise RuntimeError(
                    f"Handle mismatch (expected {handle}, got {data.get('handle')})"
                )

            return {
                "url": url,
                "status": "success",
                "data": data
            }


# =========================
# SINGLE PRODUCT WRAPPER
# =========================

async def fetch_product(session, product, semaphore):
    handle = product["handle"]
    result = await get_product_js(session, handle, semaphore)
    print(f"✅ {handle}")
    return result


# =========================
# ASYNC RUNNER
# =========================

async def fetch_all_products(all_products, concurrency=CONCURRENCY):
    semaphore = asyncio.Semaphore(concurrency)
    results = []

    async with aiohttp.ClientSession(
        headers=HEADERS,
        timeout=TIMEOUT
    ) as session:

        tasks = [
            fetch_product(session, product, semaphore)
            for product in all_products
            if "handle" in product
        ]

        for coro in asyncio.as_completed(tasks):
            try:
                result = await coro
                results.append(result)
            except Exception as e:
                print(f"❌ {e}")

    return results


# =========================
# ENTRY POINT
# =========================

if __name__ == "__main__":

    products_data = await fetch_all_products(all_products, concurrency=10)


    print(f"\nFetched {len(products_data)} products")

❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response
❌ Not JSON response


In [19]:
import requests
import time


# =========================
# CONFIG
# =========================

BASE_URL = "https://designerdaydream.com/products/"
HEADERS = {
    "User-Agent": "Mozilla/5.0"
}
TIMEOUT = 20
SLEEP_SECONDS = 1


# =========================
# PRODUCT.JS FETCHER
# =========================

def get_product_js(handle):
    """
    Fetch Shopify product JSON via:
    /products/{handle}.js
    """
    url = f"{BASE_URL}{handle}.js"

    print(f"REQUESTED: {url}")

    r = requests.get(url, headers=HEADERS, timeout=TIMEOUT)

    print(f"FINAL URL: {r.url}")

    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}")

    # if not r.headers.get("content-type", "").startswith("application/json"):
    #     raise RuntimeError("Not JSON response")

    data = r.json()

    # Safety check
    if data.get("handle") != handle:
        raise RuntimeError(
            f"Handle mismatch (expected {handle}, got {data.get('handle')})"
        )

    return {
        "url": url,
        "status": "success",
        "data": data
    }


# =========================
# SEQUENTIAL RUNNER
# =========================

def fetch_all_products(all_products):
    results = []

    for product in all_products:
        if "handle" not in product:
            continue

        handle = product["handle"]

        try:
            result = get_product_js(handle)
            results.append(result)
            print(f"✅ {handle}")
        except Exception as e:
            print(f"❌ {handle} → {e}")

        # Respect Shopify rate limits
        time.sleep(SLEEP_SECONDS)

    return results


# =========================
# ENTRY POINT
# =========================

if __name__ == "__main__":
    products_data = fetch_all_products(all_products)
    print(f"\nFetched {len(products_data)} products")

Streaming output truncated to the last 5000 lines.
REQUESTED: https://designerdaydream.com/products/versace-ve2163-aviator-sunglasses-in-gold.js
FINAL URL: https://designerdaydream.com/products/versace-ve2163-aviator-sunglasses-in-gold.js
✅ versace-ve2163-aviator-sunglasses-in-gold
REQUESTED: https://designerdaydream.com/products/versace-ve2264-oval-sunglasses-in-gold-pink.js
FINAL URL: https://designerdaydream.com/products/versace-ve2264-oval-sunglasses-in-gold-pink.js
✅ versace-ve2264-oval-sunglasses-in-gold-pink
REQUESTED: https://designerdaydream.com/products/burberry-be3074-gunmetal-polarized-aviator-sunglasses.js
FINAL URL: https://designerdaydream.com/products/burberry-be3074-gunmetal-polarized-aviator-sunglasses.js
✅ burberry-be3074-gunmetal-polarized-aviator-sunglasses
REQUESTED: https://designerdaydream.com/products/versace-ve2245-grey-square-medusa-sunglasses.js
FINAL URL: https://designerdaydream.com/products/versace-ve2245-grey-square-medusa-sunglasses.js
✅ versace-ve2245-

In [20]:
len(products_data)

1810

In [22]:
import json

file_path = 'designerdaydream_data.json'
with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(products_data, f, indent=4)
print(f"All products data saved to {file_path}")

All products data saved to designerdaydream_data.json


In [23]:
products_data[0]

{'url': 'https://designerdaydream.com/products/celine-cl4006in-thin-shadow-sunglasses-in-blue.js',
 'status': 'success',
 'data': {'id': 8188487172352,
  'title': 'Celine CL4006IN Thin Shadow Sunglasses in Blue',
  'handle': 'celine-cl4006in-thin-shadow-sunglasses-in-blue',
  'description': '<div style="white-space: pre-wrap; text-align: center;">Brand new Celine CL4006IN Thin Shadow Sunglasses.<br>In navy blue acetate with gradient blue lens.</div>\n<div style="white-space: pre-wrap; text-align: center;">\n<strong></strong>100% authentic and a classic model by Celine.</div>\n<div style="white-space: pre-wrap; text-align: center;">Full box set included.<br>Size 61-14-145.</div>\n<div style="white-space: pre-wrap; text-align: center;">Made in Italy.</div>',
  'published_at': '2025-12-24T23:49:38-05:00',
  'created_at': '2023-04-28T15:33:52-04:00',
  'vendor': 'Celine',
  'type': 'Sunglasses',
  'tags': ['celine', 'mens', 'new', 'womens'],
  'price': 20499,
  'price_min': 20499,
  'price

In [24]:
my_products = [product['data'] for product in products_data]
print(f"Created 'my_products' list with {len(my_products)} items.")

Created 'my_products' list with 1810 items.


In [25]:
# Display the first item in my_products to verify
import pprint
pprint.pprint(my_products[0])

{'available': True,
 'compare_at_price': 43000,
 'compare_at_price_max': 43000,
 'compare_at_price_min': 43000,
 'compare_at_price_varies': False,
 'created_at': '2023-04-28T15:33:52-04:00',
 'description': '<div style="white-space: pre-wrap; text-align: center;">Brand '
                'new Celine CL4006IN Thin Shadow Sunglasses.<br>In navy blue '
                'acetate with gradient blue lens.</div>\n'
                '<div style="white-space: pre-wrap; text-align: center;">\n'
                '<strong></strong>100% authentic and a classic model by '
                'Celine.</div>\n'
                '<div style="white-space: pre-wrap; text-align: center;">Full '
                'box set included.<br>Size 61-14-145.</div>\n'
                '<div style="white-space: pre-wrap; text-align: center;">Made '
                'in Italy.</div>',
 'featured_image': '//cdn.shopify.com/s/files/1/0280/3263/0877/files/CL4006IN-90B-01.jpg?v=1682710463',
 'handle': 'celine-cl4006in-thin-shadow-sun

In [27]:
import pandas as pd

BASE_URL = "https://designerdaydream.com/products/"

def flatten_products(products: list[dict]) -> pd.DataFrame:
    rows = []

    for product in products:
        product_id = product.get("id")
        product_title = product.get("title", "")
        vendor = product.get("vendor", "")
        product_type = product.get("type", "") # Corrected from 'product_type' to 'type'
        handle = BASE_URL + product.get("handle", "")
        description = product.get("description", "")

        # SAFELY resolve default image (product level image)
        images = product.get("images") or []
        default_image = images[0] if images else None # Corrected to directly use the URL string

        # Prepare default values
        option_colors = []
        option_sizes = []

        # Options can be dict OR list
        options = product.get("options", {})

        if isinstance(options, dict):
            option_colors = options.get("color", [])
            option_sizes = options.get("size", [])
        elif isinstance(options, list):
            for opt in options:
                if isinstance(opt, dict): # Added check: Ensure 'opt' is a dictionary
                    name = (opt.get("name") or "").lower()
                    if name == "color":
                        option_colors = opt.get("values", [])
                    if name == "size":
                        option_sizes = opt.get("values", [])

        for variant in product.get("variants", []):
            variant_title = variant.get("title") or product_title

            # Replace default title
            if variant_title.lower() == "default title":
                variant_title = product_title

            # Fallback to product image (corrected logic)
            variant_featured_image = variant.get("featured_image") # This will be a string URL or None
            if variant_featured_image: # Check if it's not None or empty string
                image = variant_featured_image
            else:
                image = default_image

            color = variant_title if variant_title in option_colors else None
            size = variant_title if variant_title in option_sizes else None

            # Safely get and convert price
            raw_price = variant.get("price")
            try:
                price = float(raw_price) / 100 if raw_price is not None else 0.0
            except ValueError:
                price = 0.0 # Default to 0.0 if conversion fails

            rows.append({
                "product_id": product_id,
                "product_title": product_title,
                "vendor": vendor,
                "product_type": product_type,
                "handle": handle,
                "description": description,
                "variant_id": variant.get("id"), # Changed to use 'id' as 'variant_id'
                "variant_title": variant_title,
                "sku": variant.get("sku"),
                "price": price,
                "color": color,
                "size": size,
                "available": variant.get("available"),
                "barcode": variant.get("barcode"),
                "image": image
            })

    return pd.DataFrame(rows)

df = flatten_products(my_products)

# Display as table in Colab (Excel-ready)
df.head(50)

,product_id,product_title,vendor,product_type,handle,description,variant_id,variant_title,sku,price,color,size,available,barcode,image
0,8188487172352,Celine CL4006IN Thin Shadow Sunglasses in Blue,Celine,Sunglasses,https://designerdaydream.com/products/celine-c...,"<div style=""white-space: pre-wrap; text-align:...",43726265057536,Celine CL4006IN Thin Shadow Sunglasses in Blue,CL4006IN90B,204.99,None,None,True,192337010996,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
1,9302456467712,Loewe LW40127I Anagram Sunglasses in Black,Loewe,Sunglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47501024198912,Loewe LW40127I Anagram Sunglasses in Black,LW40127I01A,304.99,None,None,True,192337156120,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
2,9302455845120,Loewe LW40129U Anagram Sunglasses in Black,Loewe,Sunglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47501022331136,Loewe LW40129U Anagram Sunglasses in Black,LW40129U01B,304.99,None,None,True,192337171826,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
3,9302453027072,Loewe LW40156U Cat Eye Sunglasses in Brown,Loewe,Sunglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47500999786752,Loewe LW40156U Cat Eye Sunglasses in Brown,LW40156U53V,269.99,None,None,True,192337199769,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
4,9302450569472,Loewe LW40154U Anagram Sunglasses in Black,Loewe,Sunglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47500972785920,Loewe LW40154U Anagram Sunglasses in Black,LW40154U01A,289.99,None,None,True,192337199653,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
5,9302447554816,Loewe LW50068I Anagram Round Frames in Transpa...,Loewe,Eyeglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47500934152448,Loewe LW50068I Anagram Round Frames in Transpa...,LW50068I51050,239.99,None,None,True,192337181085,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
6,9302445293824,Loewe LW50068I Anagram Round Frames in Black,Loewe,Eyeglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47500923535616,Loewe LW50068I Anagram Round Frames in Black,LW50068I51001,239.99,None,None,True,192337156250,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
7,9302438117632,Loewe LW50076I Anagram Frames in Black,Loewe,Eyeglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47500850430208,Loewe LW50076I Anagram Frames in Black,LW50076I52001,239.99,None,None,True,192337164835,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
8,9302434447616,Loewe LW50069I Anagram Frames in Blonde Havana,Loewe,Eyeglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47500814876928,Loewe LW50069I Anagram Frames in Blonde Havana,LW50069I51053,239.99,None,None,True,192337156335,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
9,9302430679296,Loewe LW50069I Anagram Frames in Black Horn,Loewe,Eyeglasses,https://designerdaydream.com/products/loewe-lw...,"<div style=""text-align: center;"">Brand new Loe...",47500799639808,Loewe LW50069I Anagram Frames in Black Horn,LW50069I51063,239.99,None,None,True,192337181245,//cdn.shopify.com/s/files/1/0280/3263/0877/fil...
